# Capturing and reusing JSON

This workflow takes as input a STAC Item and a STAC asset key to produce a quicklook in the PNG format.

The first step of the workflow uses `curl` to get the STAC Item json and this content is parsed to get the _assets_ element.

In pratical terms it does:

In [1]:
curl -s https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_53HPA_20210723_0_L2A | jq .assets

{
  "thumbnail": {
    "title": "Thumbnail",
    "type": "image/png",
    "roles": [
      "thumbnail"
    ],
    "href": "https://roda.sentinel-hub.com/sentinel-s2-l1c/tiles/53/H/PA/2021/7/23/0/preview.jpg"
  },
  "overview": {
    "title": "True color image",
    "type": "image/tiff; application=geotiff; profile=cloud-optimized",
    "roles": [
      "overview"
    ],
    "gsd": 10,
    "eo:bands": [
      {
        "name": "B04",
        "common_name": "red",
        "center_wavelength": 0.6645,
        "full_width_half_max": 0.038
      },
      {
        "name": "B03",
        "common_name": "green",
        "center_wavelength": 0.56,
        "full_width_half_max": 0.045
      },
      {
        "name": "B02",
        "common_name": "blue",
        "center_wavelength": 0.4966,
        "full_width_half_max": 0.098
      }
    ],
    "href": "https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2021/7/S2B_53HPA_20210723_0_L2A/L2A_PVI.tif",
    "proj:shape": 

The CWL command line tool is listed below.

The last line, `outputEval: ${ return JSON.parse(self[0].contents).assets; }`, parses the content as JSON and extracts the `assets` element.

In [2]:
yq e '.$graph[1]' workflow.cwl

class: CommandLineTool
id: stac
requirements:
  InlineJavascriptRequirement: {}
  NetworkAccess:
    networkAccess: true
hints:
  DockerRequirement:
    dockerPull: docker.io/curlimages/curl:latest
baseCommand: curl
stdout: message
arguments:
  - $( inputs.stac_item )
inputs:
  stac_item:
    type: string
outputs:
  asset:
    type: Any
    outputBinding:
      glob: message
      loadContents: true
      outputEval: ${ return JSON.parse(self[0].contents).assets; }


The second step uses the CWL expression `$( inputs.asset[inputs.asset_id].href )` to get the STAC Asset href.

This two step approach is the equivalent of:

In [3]:
curl -s https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_53HPA_20210723_0_L2A | jq .assets | jq .visual.href

"https://sentinel-cogs.s3.us-west-2.amazonaws.com/sentinel-s2-l2a-cogs/53/H/PA/2021/7/S2B_53HPA_20210723_0_L2A/TCI.tif"


The content is listed below.

It uses a CWL expression, `$( inputs.asset[inputs.asset_id].href )` to get the asset with key `asset_id` href. 

In [4]:
yq e '.$graph[2]' workflow.cwl

class: CommandLineTool
requirements:
  EnvVarRequirement:
    envDef: {}
  InlineJavascriptRequirement: {}
  NetworkAccess:
    networkAccess: true
inputs:
  asset:
    type: Any
  asset_id:
    type: string
outputs:
  quicklook:
    type: File
    outputBinding:
      glob: "quicklook.jpg"
baseCommand: rio
arguments:
  - convert
  - -f
  - JPEG
  - valueFrom: |
      $( inputs.asset[inputs.asset_id].href )
  - quicklook.jpg
hints:
  DockerRequirement:
    dockerPull: localhost/rio:latest
id: rio


This workflow how to capture JSON and then re-use it to access its elements.

## Execute the workflow

In [5]:
cwltool workflow.cwl --stac_item https://earth-search.aws.element84.com/v0/collections/sentinel-s2-l2a-cogs/items/S2B_53HPA_20210723_0_L2A --asset_id visual

INFO /opt/conda/bin/cwltool 3.1.20231114134824
INFO Resolved 'workflow.cwl' to 'file:///workspace/cwl-guide-for-eo/notebooks/03%20Capture%20JSON%20and%20reuse/workflow.cwl'
INFO [workflow ] start
INFO [workflow ] starting step step_stac
INFO [step step_stac] start
Error: error inspecting object: no such object: "docker.io/curlimages/curl:latest"
INFO ['podman', 'pull', 'docker.io/curlimages/curl:latest']
Trying to pull docker.io/curlimages/curl:latest...
Getting image source signatures
Copying blob 4ca545ee6d5d [======================================] 42.0b / 42.0b
Copying blob 4ca545ee6d5d done  
Copying blob 4ca545ee6d5d done  
Copying blob 4ca545ee6d5d done  
Copying blob 4abcf2066143 [=========================>----------] 2.3MiB / 3.3MiB
Copying blob 1113023ab841 [================>-------------------] 2.0MiB / 4.1MiB
Copying blob 4ca545ee6d5d done  
Copying blob 4abcf2066143 [====================================] 3.3MiB / 3.3MiB
Copying blob 1113023ab841 [==========================

In [6]:
 yq e '.$graph[1]' workflow.cwl

class: CommandLineTool
id: stac
requirements:
  InlineJavascriptRequirement: {}
  NetworkAccess:
    networkAccess: true
hints:
  DockerRequirement:
    dockerPull: docker.io/curlimages/curl:latest
baseCommand: curl
stdout: message
arguments:
  - $( inputs.stac_item )
inputs:
  stac_item:
    type: string
outputs:
  asset:
    type: Any
    outputBinding:
      glob: message
      loadContents: true
      outputEval: ${ return JSON.parse(self[0].contents).assets; }


In [8]:
echo "<img src="quicklook.jpg" alt="Quicklook" width="500">" | displayHTML